# Relation Extraction

In this assignment, you will work on the [MeasEval](https://competitions.codalab.org/competitions/25770) shared task that was part of SemEval-2021. The goal of **MeasEval** is  the extraction of counts, measurements, and related context from scientific documents. The task is a complex problem that involves solving a number of steps that range from identifying quantities and units of measurement to identify relationships between them. For this assignment, you will focus only on extracting the *HasQuantity* relation:

 * Given a sentence and the entities it contains, identify the *HasQuantity* relations that link *Quantity* entities with entities of other types. The task can be addressed as Pairwise Relation Extraction.

For example, the sentence in the image below contains 1 *Quantity* entity and 2 entities of other types, but only one of the latter is linked to the *Quantity* through the *HasQuantity* relation.

![image](img/hasquantity.png)

You will develop a Convolutional Neural Network with [keras](https://keras.io/), a high-level Deep Learning API written in **Python** that provides a user-friendly interface for the [TensorFlow](https://www.tensorflow.org/) library, one of the most popular low-level Deep Learning frameworks. You will use the following objects and functions:

In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import set_random_seed
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Bidirectional, TimeDistributed, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.initializers import Constant
from sklearn.metrics import f1_score

When working with Neural Networks, there are a large number of random operations such as initializing the weights of the network, shuffling the data for training, or choosing samples. This causes that different training runs of the same model can lead to different results. To ensure reproducibility, i.e. obtaining the same results in the different runs, the random number generator must be initialized with a fixed value known as seed. In **keras**, this can be done as follows:

In [4]:
seed = 42
set_random_seed(seed)

When developing a model, if the results you get are not as expected, try re-initializing the seed by running the cell above before compiling and training the model.

> **Note!** With models as complex as Neural Networks, reproducibility is susceptible to factors such as software versions or the hardware on which the models are run. Even with seed initialization, there may be slight differences in the results.

Working with Neural Networks also involves defining a number of hyperparameters that set the configuration of the model. Finding the appropriate hyperparameter values requires training the model with different combinations and testing them on the development set. This hyperparameter tuning is a costly process that needs multiple rounds of experimentation. However, for this assignments, you will use the following values:

In [5]:
maxlen = 130  # Maximum length of the input sequence accepted by the model
epochs = 10  # Number of epochs to train the model
batch_size = 64  # Number of examples used per gradient update
embedding_dim = 300  # Dimension of the embeddings
filters = 100  # Number of output filters in the convolution
kernel_size = 5  # Length of the convolution window
hidden_dim = 10  # Dimension of the hidden dense layer

Training a Deep Learning model with a large train set can be a time-consuming process, as the model needs to iterate over the entire set multiple times, often requiring significant computational resources. During the implementation of the model, it is often a good practice to use only a subset of the training data. This allows a faster debugging of the code. Set the `shrink_dataset` variable as `True` when a faster training is required and set it as `False` to train the model on the whole train set:

In [6]:
shrink_dataset = False

Although the value of this variable does not affect the tests that will evaluate your code, the output examples distributed throughout this notebook are based on a `shrink_dataset` variable set as `False`.

The train set for the assignment consists of 248 articles with 1366 sentences in total. The test set contains 136 articles with 848 sentences. A development set with 65 documents and 459 sentences is also provided. The dataset includes all the entities annotated at the token level following a BIO schema for 4 different types: *MeasuredEntity*, *MeasuredProperty*, *Qualifier* and *Quantity*. Every entity in the dataset has a unique identifier `annotId`. This identifiers are used to indicate if an entity is linked to a *Quantity* with a *HasQuantity* relation. For example, the annotation for a relation between an "*involved*" *MeasuredProperty* and a "*two beach materials*" *Quantity* would look as follows:

|       | docId                  |   sentId | word        | lemma      | label              | annotId   | rel   |
|------:|:-----------------------|---------:|:------------|:-----------|:-------------------|:----------|:------|
| 22278 | S0378383912000130-3601 |        3 | involved    | involve    | B-MeasuredProperty | T3-1      | T1-1  |
| 22279 | S0378383912000130-3601 |        3 | two         | two        | B-Quantity         | T1-1      | nan   |
| 22280 | S0378383912000130-3601 |        3 | beach       | beach      | I-Quantity         | T1-1      | nan   |
| 22281 | S0378383912000130-3601 |        3 | materials   | material   | I-Quantity         | T1-1      | nan   |

The dataset can be loaded into three `DataFrames` as follows:

In [7]:
def load_data(data_path, shrink_dataset, seed):
    data = pd.read_csv(data_path, sep="\t", encoding="utf8")
    if shrink_dataset:
        sample = data[["docId",  "sentId"]].drop_duplicates().sample(frac=0.2, random_state=seed)
        data = pd.merge(data, sample, on=["docId", "sentId"])
    return data

In [8]:
train_data = load_data("data/train_rels.tsv", shrink_dataset, seed)
dev_data = load_data("data/trial_rels.tsv", shrink_dataset, seed)
test_data = load_data("data/eval_rels.tsv", shrink_dataset, seed)
train_data[(train_data.docId == "S0378383912000130-3601") & (train_data.sentId == 3)]

,docId,sentId,word,lemma,label,annotId,rel
22276,S0378383912000130-3601,3,The,the,O,NaN,NaN
22277,S0378383912000130-3601,3,experiments,experiment,B-MeasuredEntity,T4-1,NaN
22278,S0378383912000130-3601,3,involved,involve,B-MeasuredProperty,T3-1,T1-1
22279,S0378383912000130-3601,3,two,two,B-Quantity,T1-1,NaN
22280,S0378383912000130-3601,3,beach,beach,I-Quantity,T1-1,NaN
22281,S0378383912000130-3601,3,materials,material,I-Quantity,T1-1,NaN
22282,S0378383912000130-3601,3,with,with,O,NaN,NaN
22283,S0378383912000130-3601,3,nominal,nominal,B-MeasuredProperty,T3-2,T1-2
22284,S0378383912000130-3601,3,sediment,sediment,I-MeasuredProperty,T3-2,T1-2
22285,S0378383912000130-3601,3,diameters,diameter,I-MeasuredProperty,T3-2,T1-2


The `DataFrames` created include the lemmatization of words in the `lemma` columns. You will use the lemmas as the input of the model.

> **Note!** The notebook for this assignment provides very little guidance. You are expected to refer to the [documentation](https://keras.io/api/) for details on how to solve the exercises.

## Data Pre-processing

In this assignment, you will have to implement some steps to pre-process and obtain a representation of the data. You will implement a model with an `Embedding` lookup table as the input layer, so the tokens of the input sentences should be represented as indexes. Besides, as one would expect, the sentences in the **MeasEval** dataset have different lengths. However, the input for a Deep Learning model is a batch of examples (in this case, sentences) in the form of a single tensor which requires that all examples in the batch must have the same length. Therefore, the sentences should be padded or truncated to a specific length.

> **Note!** For this particular task, the `maxlen` value provided to you guarantees that padding is sufficient to make all sentences the same length without the need for truncation. 

This first of these pre-processing steps will be to obtain both a vocabulary from the train set. The vocabulary should be the list of unique lemmas and must include the special tokens `[PAD]`, that will be used for padding the sequences, and `[UNK]`, that will be used to represent out-of-vocabulary words. The vocabulary must also include the following set of special tokens that will be use later to tag entities:

> &lt;MeasuredEntity&gt;, &lt;/MeasuredEntity&gt;, &lt;MeasuredProperty&gt;, &lt;/MeasuredProperty&gt;, &lt;Qualifier&gt;, &lt;/Qualifier&gt;, &lt;Quantity&gt;, &lt;/Quantity&gt;

Along with the vocabulary, you will also have to build a dictionary mapping each lemma to its position in the vocabulary. The dictionary will be used later to obtain the representation of the input of the model. The text is already tokenized and lemmatized which will help in this task.  The resulting vocabulary should should include the special tokens `[PAD]` and `[UNK]` in the first two positions and have 5517 items in total:
> <pre>
Vocabulary size: 5517
Vocabulary first 5 words: ['[PAD]', '[UNK]', '&lt;MeasuredEntity&gt;', '&lt;/MeasuredEntity&gt;', '&lt;MeasuredProperty&gt;', '&lt;/MeasuredProperty&gt;', '&lt;Qualifier&gt;', '&lt;/Qualifier&gt;', '&lt;Quantity&gt;', '&lt;/Quantity&gt;', 'datum', 'be', 'draw', 'from', 'the']
Vocabulary dictionary: {'[PAD]': 0, '[UNK]': 1, '&lt;MeasuredEntity>': 2, '&lt;/MeasuredEntity&gt;': 3, '&lt;MeasuredProperty&gt;': 4, '&lt;/MeasuredProperty&gt;': 5, '&lt;Qualifier&gt;': 6, '&lt;/Qualifier&gt;': 7, '&lt;Quantity&gt;': 8, '&lt;/Quantity&gt;': 9, 'datum': 10, 'be': 11, 'draw': 12, 'from': 13, 'the': 14}
</pre> 

In [9]:
def get_vocabulary(train_data):
    # Initialize vocabulary with required special tokens
    vocab = [
        '[PAD]', '[UNK]', '<MeasuredEntity>', '</MeasuredEntity>', '<MeasuredProperty>', '</MeasuredProperty>',
        '<Qualifier>', '</Qualifier>', '<Quantity>', '</Quantity>'
    ]

    word2idx = {token: idx for idx, token in enumerate(vocab)}
    unique_lemmas = list(set(train_data['lemma'].unique()))
    vocab.extend(unique_lemmas)
    word2idx.update({lemma: idx for idx, lemma in enumerate(vocab[len(word2idx):], start=len(word2idx))})
    
    return vocab, word2idx

In [10]:
vocab, word2idx, = get_vocabulary(train_data)
vocab_size = len(vocab)
print(f"Vocabulary size: {vocab_size}")
print(f"Vocabulary first 5 words: {vocab[:15]}")
print(f"Vocabulary dictionary: { {w: word2idx[w] for w in vocab[:15]}}")

Vocabulary size: 5517
Vocabulary first 5 words: ['[PAD]', '[UNK]', '<MeasuredEntity>', '</MeasuredEntity>', '<MeasuredProperty>', '</MeasuredProperty>', '<Qualifier>', '</Qualifier>', '<Quantity>', '</Quantity>', 'isotopic', 'pseudo', 'purpose', 'zno', 'child']
Vocabulary dictionary: {'[PAD]': 0, '[UNK]': 1, '<MeasuredEntity>': 2, '</MeasuredEntity>': 3, '<MeasuredProperty>': 4, '</MeasuredProperty>': 5, '<Qualifier>': 6, '</Qualifier>': 7, '<Quantity>': 8, '</Quantity>': 9, 'isotopic': 10, 'pseudo': 11, 'purpose': 12, 'zno': 13, 'child': 14}


In this assignment, you will follow a pairwise strategy for the extraction of the *HasQuantity* relation. The task must be approached by forming relation candidates by pairing all the *Quantity* entities in a sentence with the rest of entities in the sentence that belong to other entity types. For each of this pairs, the input for the model should be the text of the sentence with the entities of the pair marked using the special tags of their corresponding entity type. For example, the input for the relation candidate involving the *Quantity* "*involved*" and the *MeasuredProperty* "*two beach materials*" would look like this:

> The experiment &lt;MeasuredProperty&gt; involved &lt;/MeasuredProperty&gt; &lt;Quantity&gt; two beach materials &lt;/Quantity&gt; with nominal sediment diameter of 1.5 mm and 8.5 mm.

This strategy should result in the following number of relation candidates for each data split:

>Number of training relation candidates: 2773  
Number of development relation candidates: 797  
Number of testing relation candidates: 1445

You must reformat the train, development and test `DataFrames` by aggregating the data corresponding to each sentence. The input for the model must be the sequence of lemmas in the sentence and the output a binary True/False label indicating whether the entities of the relation candidate hold a *HasQuantity* relation.  The output of `integrate_sentences` must be a `DataFrame` with a row for each relation candidate and the following columns:

 * `docId`: the `docId` of the relation candidate
 * `sentId`: the `sentId` of the relation candidate
 * `quantityId`: the `annotId` of the *Quantity* entity of the pair
 * `otherId`: the `annotId` of the other entity of the pair
 * `lemmas`: the list of the lemmas of the sentence of the relation candidate including the special tags
 * `label`: a binary label indicating if there is a a *HasQuantity* relation between the entity pair
 
For a sentence with 2 *Quantity* entities and 3 other entities, the `DataFrame` should include 6 rows:

|      | docId                  |   sentId | quantityId   | otherId   | lemmas                                                                                                                                                                                                                        | label   |
|-----:|:-----------------------|---------:|:-------------|:----------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:--------|
| 1817 | S0378383912000130-3601 |        3 | T1-1         | T4-1      | ['the', '&lt;MeasuredEntity&gt;', 'experiment', '&lt;/MeasuredEntity&gt;', 'involve', '&lt;Quantity&gt;', 'two', 'beach', 'material', '&lt;/Quantity&gt;', 'with', 'nominal', 'sediment', 'diameter', 'of', '1.5', 'mm', 'and', '8.5', 'mm', '.']     | False   |
| 1818 | S0378383912000130-3601 |        3 | T1-1         | T3-1      | ['the', 'experiment', '&lt;MeasuredProperty&gt;', 'involve', '&lt;/MeasuredProperty&gt;', '&lt;Quantity&gt;', 'two', 'beach', 'material', '&lt;/Quantity&gt;', 'with', 'nominal', 'sediment', 'diameter', 'of', '1.5', 'mm', 'and', '8.5', 'mm', '.'] | True    |
| 1819 | S0378383912000130-3601 |        3 | T1-1         | T3-2      | ['the', 'experiment', 'involve', '&lt;Quantity>', 'two', 'beach', 'material', '&lt;/Quantity&gt;', 'with', '&lt;MeasuredProperty&gt;', 'nominal', 'sediment', 'diameter', '&lt;/MeasuredProperty>', 'of', '1.5', 'mm', 'and', '8.5', 'mm', '.'] | False   |
| 1820 | S0378383912000130-3601 |        3 | T1-2         | T4-1      | ['the', '&lt;MeasuredEntity&gt;', 'experiment', '&lt;/MeasuredEntity&gt;', 'involve', 'two', 'beach', 'material', 'with', 'nominal', 'sediment', 'diameter', 'of', '&lt;Quantity>', '1.5', 'mm', 'and', '8.5', 'mm', '&lt;/Quantity&gt;', '.']     | False   |
| 1821 | S0378383912000130-3601 |        3 | T1-2         | T3-1      | ['the', 'experiment', '&lt;MeasuredProperty&gt;', 'involve', '&lt;/MeasuredProperty&gt;', 'two', 'beach', 'material', 'with', 'nominal', 'sediment', 'diameter', 'of', '&lt;Quantity>', '1.5', 'mm', 'and', '8.5', 'mm', '&lt;/Quantity&gt;', '.'] | False   |
| 1822 | S0378383912000130-3601 |        3 | T1-2         | T3-2      | ['the', 'experiment', 'involve', 'two', 'beach', 'material', 'with', '&lt;MeasuredProperty&gt;', 'nominal', 'sediment', 'diameter', '&lt;/MeasuredProperty&gt;', 'of', '&lt;Quantity&gt;', '1.5', 'mm', 'and', '8.5', 'mm', '&lt;/Quantity&gt;', '.'] | True    |

In [46]:
def integrate_sentences(data):
    
    results = []

    quantity_entities = data[data["label"].str.contains("B-Quantity")]
    other_entities = data[data["label"].str.contains("B-MeasuredProperty|B-MeasuredEntity|B-Qualifier")]
    quantity_entities = quantity_entities.rename(columns={"annotId": "quantityId", "rel": "quantityRel", "lemma": "quantityLemma"}).reset_index()
    other_entities = other_entities.rename(columns={"annotId": "otherId", "label": "otherLabel", "lemma": "otherLemma"}).reset_index()

    # Join on docId and sentId to get all pairs within each sentence
    pairs = pd.merge(quantity_entities, other_entities, on=["docId", "sentId"])

    # Iterate through each pair to determine label based on the relationship
    for _, pair in pairs.iterrows():
        label = pair["rel"] == pair["quantityId"]

        # Extract all lemmas for the sentence with token indices
        sentence_data = data[(data["docId"] == pair["docId"]) & (data["sentId"] == pair["sentId"])].reset_index()
        sentence_lemmas = sentence_data["lemma"].tolist()

        # Tag Quantity entity in the sentence
        start_idx = sentence_data[sentence_data["index"] == pair["index_x"]].index[0]
        end_idx = start_idx
        while end_idx + 1 < len(sentence_data) and sentence_data.iloc[end_idx + 1]["label"].endswith("Quantity"):
            end_idx += 1
        sentence_lemmas[start_idx] = f"<Quantity> {sentence_lemmas[start_idx]}"
        sentence_lemmas[end_idx] = f"{sentence_lemmas[end_idx]} </Quantity>"

        # Tag Other entity in the sentence
        start_idx = sentence_data[sentence_data["index"] == pair["index_y"]].index[0]
        end_idx = start_idx
        while end_idx + 1 < len(sentence_data) and sentence_data.iloc[end_idx + 1]["label"].endswith(pair["otherLabel"].split("-")[1]):
            end_idx += 1
        label_type = pair["otherLabel"].split("-")[1]
        sentence_lemmas[start_idx] = f"<{label_type}> {sentence_lemmas[start_idx]}"
        sentence_lemmas[end_idx] = f"{sentence_lemmas[end_idx]} </{label_type}>"

        # Ensure separate tags in lemmas
        sentence_lemmas = [token for phrase in sentence_lemmas for token in phrase.split()]

        # Add the row for this pair
        row = {
            "docId": pair["docId"],
            "sentId": pair["sentId"],
            "quantityId": pair["quantityId"],
            "otherId": pair["otherId"],
            "lemmas": sentence_lemmas,
            "label": label
        }

        results.append(row)

    results = pd.DataFrame(results)

    # Drop exact duplicates in all columns to retain unique pairs
    results = results.drop_duplicates(subset=["docId", "sentId", "quantityId", "otherId"])
    results["lemmas"] = results["lemmas"].apply(tuple)  # Use tuple to ensure drop_duplicates works on lists
    results = results.drop_duplicates()
    results["lemmas"] = results["lemmas"].apply(list)  # Convert back to list

    return results


In [40]:
import pandas as pd

def integrate_sentences(data):
    relation_candidates = []

    grouped = data.groupby(['docId', 'sentId'])

    for (doc_id, sent_id), group in grouped:
        group = group.reset_index(drop=True)

        lemmas = group['lemma'].tolist()

        # Separate quantity entities and other entities
        quantity_entities = group[group['label'] == "B-Quantity"]
        other_entities = group[group['label'].isin(["B-MeasuredProperty", "B-MeasuredEntity", "B-Qualifier"])]

        for _, quantity_entity in quantity_entities.iterrows():
            quantity_annot_id = quantity_entity['annotId']

            for _, other_entity in other_entities.iterrows():
                other_annot_id = other_entity['annotId']

                lemmas_with_tags = lemmas.copy()

                # Quantity entity
                quantity_rows = group[group['annotId'] == quantity_annot_id]
                if not quantity_rows.empty:
                    start_idx = quantity_rows.index[0]
                    end_idx = start_idx
                    while (
                        end_idx + 1 < len(group)
                        and group.iloc[end_idx + 1]["label"].endswith("Quantity")
                    ):
                        end_idx += 1
                    lemmas_with_tags[start_idx] = f"<Quantity> {lemmas_with_tags[start_idx]}"
                    lemmas_with_tags[end_idx] = f"{lemmas_with_tags[end_idx]} </Quantity>"
                else:
                    print(f"Warning: Quantity annotId {quantity_annot_id} not found in group.")

                # Other entity
                other_rows = group[group['annotId'] == other_annot_id]
                if not other_rows.empty:
                    start_idx = other_rows.index[0]
                    end_idx = start_idx
                    label_type = other_entity['label'].split("-")[1]
                    while (
                        end_idx + 1 < len(group)
                        and group.iloc[end_idx + 1]["label"].endswith(label_type)
                    ):
                        end_idx += 1
                    lemmas_with_tags[start_idx] = f"<{label_type}> {lemmas_with_tags[start_idx]}"
                    lemmas_with_tags[end_idx] = f"{lemmas_with_tags[end_idx]} </{label_type}>"
                else:
                    print(f"Warning: Other annotId {other_annot_id} not found in group.")

                # Flatten nested tags into single tokens
                lemmas_with_tags = ' '.join(lemmas_with_tags).split()
                # Apply consistent format for each token in lemmas_with_tags
                lemmas_with_tags = [token for token in lemmas_with_tags]

                # HasQuantity relation
                label = (quantity_entity['annotId'] == other_entity['rel'])

                relation_candidates.append({
                    'docId': doc_id,
                    'sentId': sent_id,
                    'quantityId': quantity_annot_id,
                    'otherId': other_annot_id,
                    'lemmas': lemmas_with_tags,
                    'label': label
                })

    relation_candidates = pd.DataFrame(relation_candidates)
    # Drop exact duplicates in all columns to retain unique pairs
    relation_candidates = relation_candidates.drop_duplicates(subset=['docId', 'sentId', 'quantityId', 'otherId'])
    relation_candidates['lemmas'] = relation_candidates['lemmas'].apply(tuple)  # Use tuple to ensure drop_duplicates works on lists
    relation_candidates = relation_candidates.drop_duplicates()
    relation_candidates['lemmas'] = relation_candidates['lemmas'].apply(list)  # Convert back to list

    return relation_candidates

In [47]:
train_examples = integrate_sentences(train_data)
dev_examples = integrate_sentences(dev_data)
test_examples = integrate_sentences(test_data)
pd.set_option('display.max_colwidth', None)
display(train_examples[(train_examples.docId == "S0378383912000130-3601") & (train_examples.sentId == 3)])
print("Number of training relation candidates: %s" % len(train_examples))
print("Number of development relation candidates: %s" % len(dev_examples))
print("Number of testing relation candidates: %s" % len(test_examples))

,docId,sentId,quantityId,otherId,lemmas,label
1817,S0378383912000130-3601,3,T1-1,T4-1,"[the, <MeasuredEntity>, experiment, </MeasuredEntity>, involve, <Quantity>, two, beach, material, </Quantity>, with, nominal, sediment, diameter, of, 1.5, mm, and, 8.5, mm, .]",False
1818,S0378383912000130-3601,3,T1-1,T3-1,"[the, experiment, <MeasuredProperty>, involve, </MeasuredProperty>, <Quantity>, two, beach, material, </Quantity>, with, nominal, sediment, diameter, of, 1.5, mm, and, 8.5, mm, .]",True
1819,S0378383912000130-3601,3,T1-1,T3-2,"[the, experiment, involve, <Quantity>, two, beach, material, </Quantity>, with, <MeasuredProperty>, nominal, sediment, diameter, </MeasuredProperty>, of, 1.5, mm, and, 8.5, mm, .]",False
1820,S0378383912000130-3601,3,T1-2,T4-1,"[the, <MeasuredEntity>, experiment, </MeasuredEntity>, involve, two, beach, material, with, nominal, sediment, diameter, of, <Quantity>, 1.5, mm, and, 8.5, mm, </Quantity>, .]",False
1821,S0378383912000130-3601,3,T1-2,T3-1,"[the, experiment, <MeasuredProperty>, involve, </MeasuredProperty>, two, beach, material, with, nominal, sediment, diameter, of, <Quantity>, 1.5, mm, and, 8.5, mm, </Quantity>, .]",False
1822,S0378383912000130-3601,3,T1-2,T3-2,"[the, experiment, involve, two, beach, material, with, <MeasuredProperty>, nominal, sediment, diameter, </MeasuredProperty>, of, <Quantity>, 1.5, mm, and, 8.5, mm, </Quantity>, .]",True


Number of training relation candidates: 2773
Number of development relation candidates: 797
Number of testing relation candidates: 1445


The dataset is now ready for you to get the numerical representation of the input. You must perform two steps to process the sequence of lemmas:
1. For each sentence, translate each lemma or label to its corresponding index using the `word2idx` and `label2idx` dictionaries. In case the lemma is not found in `word2idx`, use the index of the `[UNK]` token instead.
2. Pad both the sequences of lemmas and the sequences of labels to the same length as defined by the `maxlen` variable using the index of the `[PAD]` token in the vocabulary.   

You must also return the target labels in a separate array.

Applying `format_examples` to the train, development and test sets should result on 6 arrays with the following shapes:

>Shape of training input data:  (2773, 130)  
Shape of training output data:  (2773,)  
Shape of development input data:  (797, 130)  
Shape of development output data:  (797,)  
Shape of testing input data:  (1445, 130)  
Shape of testing output data:  (1445,)  

In [129]:
def format_examples(data, word2idx, maxlen):

    x_data = []
    y_data = []

    for lemmas, labels in zip(data['lemmas'], data['label']):
        lemma_indices = [word2idx.get(lemma, word2idx['[UNK]']) for lemma in lemmas]
        label_indices = 1 if labels else 0

        x_data.append(lemma_indices)
        y_data.append(label_indices)

    x_data = pad_sequences(x_data, maxlen=maxlen)
    y_data = np.array(y_data)

    return x_data, y_data

In [130]:
x_train, y_train = format_examples(train_examples, word2idx, maxlen)
x_dev, y_dev = format_examples(dev_examples, word2idx, maxlen)
x_test, y_test = format_examples(test_examples, word2idx, maxlen)
print("Shape of training input data: ", x_train.shape)
print("Shape of training output data: ", y_train.shape)
print("Shape of development input data: ", x_dev.shape)
print("Shape of development output data: ", y_dev.shape)
print("Shape of testing input data: ", x_test.shape)
print("Shape of testing output data: ", y_test.shape)

Shape of training input data:  (2773, 130)
Shape of training output data:  (2773,)
Shape of development input data:  (797, 130)
Shape of development output data:  (797,)
Shape of testing input data:  (1445, 130)
Shape of testing output data:  (1445,)


## Convolutional Neural Network

You will construct a CNN as a **keras** `Sequential` model with the following 5 layers:

1. An `Input` layer where the length of the input sequences is set to `maxlen`.
1. An `Embedding` layer with an input dimension equal to the vocabulary size and an embedding dimension defined by `embedding_dim`.
2. A `Conv1D` layer with a number of output filters determined by `filters`, a length of convolution window equal to `kernel_size` and a `relu` activation function.
3. A `GlobalMaxPooling1D` to downsample the output of the convolutional layer.
4. A `Dense` layer with a number of units equal to `hidden_dim` and a `relu` activation function.
5. A `Dense` layer with 1 unit and a `sigmoid` activation function.

Any option not mentioned in the description should be kept with its default value. The summary of the resulting model should look like:


> <pre>
> Model: "sequential"
> __________________________________________________________________________________________
>  Layer (type)                           Output Shape                        Param #       
> ==========================================================================================
>  embedding (Embedding)                  (None, 130, 300)                    1655100       
>                                                                                           
>  conv1d (Conv1D)                        (None, 126, 100)                    150100        
>                                                                                           
>  global_max_pooling1d (GlobalMaxPooling  (None, 100)                        0             
>  1D)                                                                                      
>                                                                                           
>  dense (Dense)                          (None, 10)                          1010          
>                                                                                           
>  dense_1 (Dense)                        (None, 1)                           11            
>                                                                                           
> ==========================================================================================
> Total params: 1,806,221
> Trainable params: 1,806,221
> Non-trainable params: 0
> __________________________________________________________________________________________
> </pre>

Before returning the model, you should compile it using `'binary_crossentropy'` as the loss function, `'adam'` as the optimizer and `'binary_accuracy'` as a metric to evaluate the model during training

In [136]:
def create_model(vocab_size, maxlen, embedding_dim, filters, kernel_size, hidden_dim):
   # Define model 
    model = Sequential([
        Input(shape=(maxlen,)),
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(hidden_dim, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # Compile model
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['binary_accuracy']
    )
    
    return model

In [137]:
model = create_model(vocab_size, maxlen, embedding_dim, filters, kernel_size, hidden_dim)
model.summary(line_length=90)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                          ┃ Output Shape                 ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                 │ (None, 130, 300)             │       1,655,100 │
├───────────────────────────────────────┼──────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                       │ (None, 126, 100)             │         150,100 │
├───────────────────────────────────────┼──────────────────────────────┼─────────────────┤
│ global_max_pooling1d                  │ (None, 100)                  │               0 │
│ (GlobalMaxPooling1D)                  │                              │                 │
├───────────────────────────────────────┼──────────────────────────────┼─────────────────┤
│ dense (Dense)                         │ (None, 10)                   │           1,010 │
├───────────────────────────────────────┼──────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                       │ (None, 1)                    │              11 │
└───────────────────────────────────────┴──────────────────────────────┴─────────────────┘

 Total params: 1,806,221 (6.89 MB)

 Trainable params: 1,806,221 (6.89 MB)

 Non-trainable params: 0 (0.00 B)

Once the data has been processed and the model has been compiled, you can proceed to train it using the train input and output obtained by `format_examples` as well as the development input and output produced by the same function. Yould should also use the `batch_size` and `epochs` hyperparameters and evaluate the loss and any model metrics on the development data during the training.

In [138]:
def train_model(model, x_train, y_train, x_dev, y_dev, batch_size, epochs):
    
    history = model.fit(
        x_train, y_train,
        validation_data=(x_dev, y_dev),
        batch_size=batch_size,
        epochs=epochs,
    )
    
    return history

In [139]:
train_model(model, x_train, y_train, x_dev, y_dev, batch_size, epochs)

Epoch 1/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - binary_accuracy: 0.6578 - loss: 0.6288 - val_binary_accuracy: 0.7792 - val_loss: 0.4734
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - binary_accuracy: 0.7954 - loss: 0.4371 - val_binary_accuracy: 0.8695 - val_loss: 0.3240
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 95ms/step - binary_accuracy: 0.9067 - loss: 0.2441 - val_binary_accuracy: 0.8846 - val_loss: 0.2792
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - binary_accuracy: 0.9500 - loss: 0.1484 - val_binary_accuracy: 0.8858 - val_loss: 0.2804
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - binary_accuracy: 0.9689 - loss: 0.0966 - val_binary_accuracy: 0.8695 - val_loss: 0.3112
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - binary_accuracy: 0.9765 - loss: 0.0682 - val_binary_accuracy: 0.8507 - val_loss: 0.3464
Epoch 7/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - binary_accuracy: 0.9812 - loss: 0.0519 - val_binary_accuracy: 0.8745 - val_loss: 0.3355
Epoch 8/10
44

After training, you can apply the model to make predictions on  the test input data produced by `format_examples` using the `batch_size` hyperparameter. Since the output layer of the model is a `sigmoid` function, it will return values ranging from 0 to 1. For each prediction, you should obtain a boolean `True` label in case the prediction is higher than `0.5` or `False` otherwise.

In [148]:
def make_predictions(model, x_test, batch_size):
    
    pred = model.predict(x_test, batch_size=batch_size).flatten()
    pred_labels = pred > 0.5

    return pred_labels

In [149]:
test_examples['prediction'] = make_predictions(model, x_test, batch_size)
test_examples[(test_examples.docId == "S0038071711004354-1624") & (test_examples.sentId == 2)]

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step


,docId,sentId,quantityId,otherId,lemmas,label,prediction
489,S0038071711004354-1624,2,T1-5,T4-5,"[<Quantity>, approximately, 15, min, </Quantity>, <Qualifier>, before, the, beginning, of, the, experiment, </Qualifier>, ,, the, aboveground, vegetation, be, remove, under, the, dual, -, chamber, ,, as, well, as, the, reference, chamber, which, be, deploy, at, 30, cm, distance, from, the, dual, -, chamber, .]",False,False
490,S0038071711004354-1624,2,T1-5,T2-5,"[<Quantity>, approximately, 15, min, </Quantity>, before, the, beginning, of, the, experiment, ,, the, <MeasuredEntity>, aboveground, vegetation, </MeasuredEntity>, be, remove, under, the, dual, -, chamber, ,, as, well, as, the, reference, chamber, which, be, deploy, at, 30, cm, distance, from, the, dual, -, chamber, .]",False,False
491,S0038071711004354-1624,2,T1-5,T3-5,"[<Quantity>, approximately, 15, min, </Quantity>, before, the, beginning, of, the, experiment, ,, the, aboveground, vegetation, be, <MeasuredProperty>, remove, under, the, dual, -, chamber, </MeasuredProperty>, ,, as, well, as, the, reference, chamber, which, be, deploy, at, 30, cm, distance, from, the, dual, -, chamber, .]",True,True
492,S0038071711004354-1624,2,T1-5,T4-6,"[<Quantity>, approximately, 15, min, </Quantity>, before, the, beginning, of, the, experiment, ,, the, aboveground, vegetation, be, remove, under, the, dual, -, chamber, ,, as, well, as, the, <MeasuredEntity>, reference, chamber, </MeasuredEntity>, which, be, deploy, at, 30, cm, distance, from, the, dual, -, chamber, .]",False,True
493,S0038071711004354-1624,2,T1-5,T3-6,"[<Quantity>, approximately, 15, min, </Quantity>, before, the, beginning, of, the, experiment, ,, the, aboveground, vegetation, be, remove, under, the, dual, -, chamber, ,, as, well, as, the, reference, chamber, which, be, deploy, at, 30, cm, <MeasuredProperty>, distance, from, the, dual, -, chamber, </MeasuredProperty>, .]",False,False
494,S0038071711004354-1624,2,T1-6,T4-5,"[approximately, 15, min, <Qualifier>, before, the, beginning, of, the, experiment, </Qualifier>, ,, the, aboveground, vegetation, be, remove, under, the, dual, -, chamber, ,, as, well, as, the, reference, chamber, which, be, deploy, at, <Quantity>, 30, cm, </Quantity>, distance, from, the, dual, -, chamber, .]",False,False
495,S0038071711004354-1624,2,T1-6,T2-5,"[approximately, 15, min, before, the, beginning, of, the, experiment, ,, the, <MeasuredEntity>, aboveground, vegetation, </MeasuredEntity>, be, remove, under, the, dual, -, chamber, ,, as, well, as, the, reference, chamber, which, be, deploy, at, <Quantity>, 30, cm, </Quantity>, distance, from, the, dual, -, chamber, .]",False,False
496,S0038071711004354-1624,2,T1-6,T3-5,"[approximately, 15, min, before, the, beginning, of, the, experiment, ,, the, aboveground, vegetation, be, <MeasuredProperty>, remove, under, the, dual, -, chamber, </MeasuredProperty>, ,, as, well, as, the, reference, chamber, which, be, deploy, at, <Quantity>, 30, cm, </Quantity>, distance, from, the, dual, -, chamber, .]",False,True
497,S0038071711004354-1624,2,T1-6,T4-6,"[approximately, 15, min, before, the, beginning, of, the, experiment, ,, the, aboveground, vegetation, be, remove, under, the, dual, -, chamber, ,, as, well, as, the, <MeasuredEntity>, reference, chamber, </MeasuredEntity>, which, be, deploy, at, <Quantity>, 30, cm, </Quantity>, distance, from, the, dual, -, chamber, .]",False,True
498,S0038071711004354-1624,2,T1-6,T3-6,"[approximately, 15, min, before, the, beginning, of, the, experiment, ,, the, aboveground, vegetation, be, remove, under, the, dual, -, chamber, ,, as, well, as, the, reference, chamber, which, be, deploy, at, <Quantity>, 30, cm, </Quantity>, <MeasuredProperty>, distance, from, the, dual, -, chamber, </MeasuredProperty>, .]",True,True


Since the goal of the task is the extraction of the *HasQuality* relation, the metric used for evaluation should only report results for the `True` cases. For this, `binary f1` can be used. For the model trained above, the result of this evaluation should look like:

> binary f1: 0.771

In [ ]:
def evaluate(data):
    print("binary f1: %.3f" % f1_score(data['label'], data['prediction'], average='binary'))

In [ ]:
evaluate(test_examples)